In [1]:
import os
from maze_env import Maze
from utility import Prediction
import time

In [2]:
def onKeyPress(event):
    is_terminate = False
    if (event.char == "w"):
        is_terminate = env.take_action(0, show_animate=True)
    elif (event.char == "d"):
        is_terminate = env.take_action(1, show_animate=True)
    elif (event.char == "s"):
        is_terminate = env.take_action(2, show_animate=True)
    elif (event.char == "a"):
        is_terminate = env.take_action(3, show_animate=True)
    elif (event.char == "q"):
        env.destroy()
    if (is_terminate):
        env.reset()

In [3]:
# if __name__ == "__main__":
env = Maze()
env.bind('<KeyPress>', onKeyPress)
env.mainloop()

In [4]:
print('{0}x{1}, act_space = {2}'.format(env.MAZE_Limit[0],env.MAZE_Limit[1], len(env.action_space)))
prd = Prediction(env)
prd.iteration(update = 10)

4x4, act_space = 4
0 [[ 0. -1. -1. -1.]
 [-1. -1. -1. -1.]
 [-1. -1.  0. -1.]
 [-1. -1. -1.  0.]]
1 [[ 0. -1. -2. -2.]
 [-1. -2. -2. -2.]
 [-2. -2.  0. -1.]
 [-2. -2. -1.  0.]]
2 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3.  0. -1.]
 [-3. -2. -1.  0.]]
3 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3.  0. -1.]
 [-3. -2. -1.  0.]]
4 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3.  0. -1.]
 [-3. -2. -1.  0.]]
5 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3.  0. -1.]
 [-3. -2. -1.  0.]]
6 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3.  0. -1.]
 [-3. -2. -1.  0.]]
7 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3.  0. -1.]
 [-3. -2. -1.  0.]]
8 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3.  0. -1.]
 [-3. -2. -1.  0.]]
9 [[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3.  0. -1.]
 [-3. -2. -1.  0.]]


In [5]:
import numpy as np
#testing field
p = env.get_env_model(); print("p",p.shape)
v = prd.value; print("v",v.shape)
r = prd.reward; print("r",r.shape)

p (4, 16, 16)
v (16,)
r (4, 16)


In [6]:
np.max(r + prd.discont_factor*np.matmul(p,v), axis = 0)

array([ 0., -1., -2., -3., -1., -2., -3., -2., -2., -3.,  0., -1., -3.,
       -2., -1.,  0.])